# SCC.413 Applied Data Mining
# NLP: Week 16
# Web Scraping

## Contents
- [Introduction](#intro)
- [Packages & imports](#packages)
- [Justext](#justext)
- [Requests](#requests)
- [Beautiful Soup](#beautiful_soup)
- [Wikipedia exercise](#wiki_assessed)
- [Optional Wikipedia exercise](#wiki_optional)
- [Forum exercise](#forum)
- [Advanced Forum exercise](#forum_advanced)

<a name="intro"></a>
## Introduction

If you use spidering or just download a list of webpage URLs (e.g. with [curl](https://curl.haxx.se/docs/manpage.html) or [requests](https://requests.readthedocs.io/)), you will be left with a collection of HTML files. These raw HTML files contain a lot of redundant information (e.g. adverts, menus, headings, etc.), this is known as "boilerplate". What we want is the main text of the webpage only (i.e. the news article, the blog post, etc.), in plain text without the HTML tags. If you are not familiar with HTML, [there is a guide here](https://www.w3schools.com/html/).

<a name="packages"></a>
## Packages & Imports

The packages required for the week's lab work are already available on Colab, except for justext. Running the cell below will install justext on Colab.

In [ ]:
!pip install justext

Non-standard packages are also included in `requirements.txt`, if you need to install them on your own machine.

The imports for all of the code in this lab are provided in one cell here for convenience.

In [ ]:
import json
import justext
import requests
from urllib.parse import urljoin #https://docs.python.org/3/library/urllib.parse.html#urllib.parse.urljoin
from bs4 import BeautifulSoup, Tag #https://www.crummy.com/software/BeautifulSoup/bs4/doc/
import re

<a name="justext"></a>
## Justext

Webpages come in all shapes and sizes, and it varies massively how easy it is to scrape the text of interest. If we are lucky, it is relatively easy to pick out the text, and there are fully automated tools available for that. One of these tools is [justext](https://github.com/miso-belica/jusText).

1. Have a read through the [description of the justext algorithm](http://corpus.tools/wiki/Justext/Algorithm).
2. There is a good [online demo of the tool](https://nlp.fi.muni.cz/projects/justext/). Try out a webpage, e.g. a BBC news article, this article demonstrates it well: <http://www.bbc.co.uk/bbcthree/article/cc72247b-e658-4af8-a838-dfe4e68e2776>.
3. Manually compare the filtered text and the text on the web page. How accurate is the text extraction? How much is missed, how much text is incorrectly included. What are the potential impacts of this in a later analysis of the text?
4. You can use justext via Python too. The HTML needs to be downloaded from a website, or using a file already collected. The `print_justext` function below takes some html and prints out the justext output.



In [ ]:
def print_justext(html):
  paragraphs = justext.justext(html, justext.get_stoplist("English"))
  for paragraph in paragraphs:
    if not paragraph.is_boilerplate:
      tag = 'h' if paragraph.heading else 'p'
      print("<{}> {}".format(tag, paragraph.text))

5. You can use this function to process a previously gathered webpage, or a sample webpage is available alongside this notebook (/content/5.html), as shown below.
6. Examine the text produced. How much of the text is correctly outputted?
7. Note the `<h>` and `<p>` tags, indicating headers and paragraphs. These may be useful, if for instance you are only interested in headers or the actual body of the text, or want to separate them in later analysis. You'll learn next week how you could filter these, e.g. using regular expressions.

In [ ]:
with open("/content/5.html", "r") as html_file:
  html = html_file.read()
  print_justext(html)

Often an automated approach will not provide accurate enough results. Fortunately, there are other methods available, other than just manually copying and pasting the text. Many tools have been built that assist in parsing web pages and extracting the text of interest, although scripts need writing using these tools for different sets of websites. Some mimic a user's interaction with a website to get to the relevant data (e.g. [Selenium](http://seleniumhq.github.io/selenium/docs/api/py/)). [Scrapy](https://doc.scrapy.org/en/latest/index.html) is another good option for grabbing and processing webpages. 

<a name="requests"></a>
## Requests

Here we will be using the Python requests package, which makes downloading webpages easy: <https://requests.readthedocs.io/>.

We simply provide a URL. This can be fed into justext, as below.

In [ ]:
response = requests.get("http://www.bbc.co.uk/bbcthree/article/cc72247b-e658-4af8-a838-dfe4e68e2776")
print_justext(response.text)

<a name="beautiful_soup"></a>
## Beautiful Soup

Requests will provide raw HTML files. The key part of web scraping is extracting the relevant parts of the webpage. For this we will use Beautiful Soup: <https://www.crummy.com/software/BeautifulSoup/bs4/doc/>

The basic process is to look at the HTML of the target webpage and look for ways of drilling down to the elements of interest, with the overall aim of extracting just the specific text of interest. This could be metadata, or actual text for further analysis. There are numerous methods provided by Beautiful Soup, please consult [the documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) for options available.

All websites are different (although some have consistent structures), and often a custom scraper needs to be developed. You can use a standard web browser to look at the information of interest, right-click on the first part of the data of interest and select "Inspect Element" (Firefox & Safari) or "Inspect" (Chrome), you will then see the HTML code for that element, and the surrounding elements.

You can then use Beautiful Soup's functions for drilling down and traversing the relevant parts of the web page. You can also extract links and use the requests package to download further webpages for processing.

To demonstrate, we will be parsing Wikipedia pages. As an example we will look to extract plot summaries for Star Trek episodes: <https://en.wikipedia.org/wiki/List_of_Star_Trek:_The_Original_Series_episodes>. We download the webpage using requests, and then use Beautiful Soup to put the html into a parseable document.

In [ ]:
base_url = "https://en.wikipedia.org/wiki/List_of_Star_Trek:_The_Original_Series_episodes"
#load webpage
req = requests.get(base_url)
soup = BeautifulSoup(req.text, "html.parser")

Use a web browser to view the [Wikipedia page](https://en.wikipedia.org/wiki/List_of_Star_Trek:_The_Original_Series_episodes). We are targetting the list of episodes in the tables starting under "Season 1 (1966–67)". Right click on the table cell with the first episode title ("The Man Trap") and "Inspect Element" (or just "Inspect" in Chrome). Note that this cell (`td`) is of class "summary", as is every title cell, and other cells are not. This is our way in. We are going to collect a list of titles from the table, along with the URL of the Wikipedia page about that episode.

In [ ]:
episodes = [] #to store the list of episodes.
#find and loop through all tds (table cells) with class name ``summary'' (which we know is an episode title)
for episode_cell in soup.find_all('td', {'class': 'summary'}):
    title = episode_cell.a.text.strip() #Get the actual text from the cell.
    episode_url = episode_cell.a['href'] #extract the url
    episodes.append({'title': title, 'url': episode_url}) #store in dictionary format
    
episodes

Note the URLs are relative, they can be made absolute with urljoin, using the base url as a reference:

In [ ]:
from urllib.parse import urljoin #https://docs.python.org/3/library/urllib.parse.html#urllib.parse.urljoin
for episode in episodes:
    episode['url'] = urljoin(base_url, episode['url'])
    
episodes

Now we have a list of episodes and their individual wikipedia pages for downloading. We can try justext on the first episode to see if automatic extraction will do:

In [ ]:
episode_req = requests.get(episodes[0]['url']) #use requests to download episode webpage.

paragraphs = justext.justext(episode_req.text, justext.get_stoplist("English"))
text = ""

for p in paragraphs:
    if not p.is_boilerplate:
        text += p.text.strip() + "\n"
        
print(text)

Compare this to the [Wikipedia page](https://en.wikipedia.org/wiki/The_Man_Trap). It does a pretty good job of getting the text from the whole page, however, we want to only include the plot section. To do this, we need to be more specific about what to extract, so we can use Beautiful Soup.

Looking at the [Wikipedia page](https://en.wikipedia.org/wiki/The_Man_Trap), you can see the sections are headed with an `h2` element, so to find the "Plot" section we just need to go through the `h2` tags, find the one with "Plot" and hoover up all of the text between there and the next section. We can do this for the first episode as follows:

In [ ]:
from bs4 import Tag #we need the Tag class from Beautiful Soup to check if a node we are looking at is Tag.

episode_soup = BeautifulSoup(episode_req.text, "html.parser") #use beautiful soup to decode into a parseable document.

episode_plot = ""

for h2 in episode_soup.find_all("h2"): #Go through all of the h2 elements.
            if(h2.text.strip().startswith("Plot")): #This is the h2 With "Plot" (and "Plot Summary")
                node = h2.next_sibling #start looking for tags after the Plot h2, will be strings and Tags.
                while True:
                    if isinstance(node, Tag): #Check if this element is actually a Tag.
                        if node.name == "p": #p tag, we want this.
                            episode_plot += node.text.strip() + "\n" #append the text from p.
                        elif node.name == "h2": #at the next h2, so a new section, no longer the plot. Stop processing.
                            break
                    node = node.next_sibling #get next element at same level.
                    
print(episode_plot)

You'll see we now just have the plot text. We just need to wrap this up in a loop and add the plot to each episode.

In [ ]:
for episode in episodes:
    episode_plot = ""
    episode_req = requests.get(episode['url']) #do a new request for the episode page.
    episode_soup = BeautifulSoup(episode_req.text, "html.parser") #use beautiful soup to decode into a parseable document.
    for h2 in episode_soup.find_all("h2"): #Go through all of the h2 elements.
        if(h2.text.strip().startswith("Plot")): #This is the h2 With "Plot" (and "Plot Summary")
            node = h2.next_sibling #start looking for tags after the Plot h2, will be strings and Tags.
            while True:
                if isinstance(node, Tag): #Check if this element is actually a Tag.
                    if node.name == "p": #p tag, we want this.
                        episode_plot += node.text.strip() + "\n" #append the text from p.
                    elif node.name == "h2": #at the next h2, so a new section, no longer the plot. Stop processing.
                        break
                node = node.next_sibling #get next element at same level.

        episode['plot'] = episode_plot #add the plot to episode.




As this is in dictionary format, it's nice to convert to JSON:



In [ ]:
print(json.dumps(episodes,indent=4)) #print out the resulting json (pretty printed).

And the output can be saved to a file:

In [ ]:
with open('startrek.json', 'w') as f:
  #Dump json file. indent=4 prints the output prettier, but will increase disk space.
  json.dump(episodes, f, indent=4)

The code is split up here to explain it more neatly in a notebook, the the full code is available in `startrekscrape.py`.

<a name="wiki_assessed"></a>

## Exercise 1: Wikipedia scraping

To practice using Beautiful Soup, try extracting details of films by Stanley Kubrick from this page: <https://en.wikipedia.org/wiki/Filmography_and_awards_of_Stanley_Kubrick>. The aim is to extract the year and title for the 13 feature films listed. To make it a little trickier, try only outputting films with Kubrick as a Producer. Some starting code is provided below.

Hint: You can use [findAll with a limit](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#the-limit-argument) to return only a limited number of nodes from another node, e.g. `td`s in a `tr`.

In [ ]:
# Exercise 1: Wikipedia scraping

base_url = "https://en.wikipedia.org/wiki/Filmography_and_awards_of_Stanley_Kubrick"
#load webpage
req = requests.get(base_url)
#Use beautiful soup to decode webpage text into parseable document.
soup = BeautifulSoup(req.text, "html.parser")

table = soup.find('table', {'class': 'wikitable'}) #just find 1 table, as it's the first 1
trs = table.findAll('tr') #all table rows
trs = trs[1:] #skip header row



<a name="wiki_optional"></a>
### Optional Wikipedia Exercise

You can also use what you've learnt to parse details of something else from Wikipedia, e.g. other TV series, or albums from an artist's discography.

In [ ]:
# Optional Task Wikipedia



<a name="forum"></a>
## Exercise 2: Forum scraping

The optional lab workbook uses Scrapy to download pages from a forum, you can perform a similar task for forums with Requests and Beautiful Soup. You can choose any forum and use what you've learnt to parse thread posts into plain text. Start with an individual thread, and one page of posts within that thread. A good example to try is from Mumsnet on noisy baby toys: <https://www.mumsnet.com/Talk/toys_and_games_chat/3414974-noisy-baby-toys-which-are-the-worst>.

1. Try using justext to parse the the first page of posts, examine the results. Is this good enough?
2. Use Beautiful Soup to collect just the posts from the first page and output as plain text.

Hint: one issue you may come across is `<br>` tags instead of newlines. If ignored, this will run lines of text in the same post directly next to each other (without even a space), e.g. something horrible like this:
"Allllllll of the toot tootSpin and bounce zebraAnd best (worst!) of all... peppa pig alphaphonic board- it has no off switch and the slightest touch sets it off for ages..."

This makes tokenisation difficult and is tricky to rectify. This is a good lesson in sanity checking the exported text, better to have the text as close to as appears on the webpage now. To deal with this issue, the `<br>` tags can be replaced with a new line character (or some other marker), e.g.:

```
for br in post.find_all("br"):
    br.replace_with("\n")
```

In [ ]:
# Exercise 2: Forum scraping



<a name="forum_advanced"></a>
## Advanced Exercise: Forum paging

You will notice that the topic's posts are spread across multiple pages, this means that "paging" needs to be performed to extract the posts from every page. This is a little tricky, but work from collecting 1 page. You will need to consult the [requests documentation](https://requests.readthedocs.io/) to discover how to pass parameters to match the links to other pages. Be careful not to have duplicate posts in your extraction (the original post is repeated on each page).

The trickiest part is to know when to stop. Going past the number of pages available just brings the user to the final page. We will be covering regular expressions next week, so to help you out, the following code can be used to find the final page.

In [ ]:
#this isn't complete code, you'll need to incorporate it into your solution.

lastpage = re.compile(r"page (\d+) of \1") #compiled regular expression, checking if we're at page n of n, i.e. last page.

#...

pages = soup.find('div', {'class': 'pages'}).text.strip() #find pages element which has the "This is page x of y" text.
if lastpage.search(pages) != None: #if our regex matches, then we're on last page, so make this the last one to parse.
    at_last = True

In [ ]:
# Advanced Exercise: Forum paging



You can go further by parsing multiple threads from a (page of a) subforum (or "Talk" on Mumsnet), e.g. try extracting all of the threads from the first page from baby names discussions: <https://www.mumsnet.com/Talk/baby_names>.

In [ ]:
# Advanced Exercise: Multiple threads

